In [1]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from mord import LogisticIT
import matplotlib.pylab as plt
import seaborn as sns
from dmba import classificationSummary, gainsChart, liftChart
from dmba.metric import AIC_score
import math

no display found. Using non-interactive Agg backend


In [2]:
banks = pd.read_csv('banks.csv')

In [3]:
banks.head()

,Obs,Financial Condition,TotCap/Assets,TotExp/Assets,TotLns&Lses/Assets
0,1,1,9.7,0.12,0.65
1,2,1,1.0,0.11,0.62
2,3,1,6.9,0.09,1.02
3,4,1,5.8,0.10,0.67
4,5,1,4.3,0.11,0.69


In [4]:
banks.shape

(20, 5)

In [5]:
banks['Financial Condition'].unique()

array([1, 0], dtype=int64)

In [6]:
predictors = ['TotExp/Assets', 'TotLns&Lses/Assets']
outcome = ['Financial Condition']

In [7]:
X = banks[predictors]
y = banks[outcome]

In [8]:
#partition data
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size = 0.4, random_state = 1)

#fit a logistic regression(set penalty=l2 and C=1e42 to avoid regularization)
logit_reg = LogisticRegression(penalty = 'l2', C = 1e42, solver = 'liblinear')
logit_reg.fit(train_X, train_y)

C:\Users\GETHSEMAN\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression(C=1e+42, solver='liblinear')

In [9]:
print('intercept ', logit_reg.intercept_[0])
print(pd.DataFrame({'coeff': logit_reg.coef_[0]}, index = X.columns).transpose())

intercept  -1115.834670153206
       TotExp/Assets  TotLns&Lses/Assets
coeff   15397.963729         -922.752711


Logit(Financial Condition = Weak) 
  = -1115.8346 + 15397.9637TotExp/Assets - 922.7527TotLns&Lses/Assets

Odds(Financial Condition = Weak|TotExp/Assets = x|TotLns&Lses/Assets = y) 
  =  e^-1115.8346 + 15397.9637x - 922.7527y

P(Financial Condition = Weak|TotExp/Assets = x|TotLns&Lses/Assets = y) 
  = 1/(1+e^-(-1115.8346 + 15397.9637x - 922.7527y)

In [10]:
test_data = pd.DataFrame({'TotExp/Assets':0.11, 'TotLns&Lses/Assets': 0.6}, index=[0] )

In [21]:
logit  = -1115.8346 + (15397.9637 * 0.11) - (922.7527 * 0.6)
print("The logit of being financially weak is: %.4f"% logit)

The logit of being financially weak is: 24.2898


In [23]:
odds = math.exp(logit)
print("The odds of being financially weak is: %.4f"% odds)

The odds of being financially weak is: 35393251382.5086


In [26]:
Prob = 1/(1 + math.exp(-exponent))
print("The probability of being financially weak is: %.4f"% Prob)

The probability of being financially weak is: 1.0000


In [14]:
bank_pred = logit_reg.predict(test_data)
bank_pred

array([1], dtype=int64)

The cutoff value of 0.5 is used in conjunction with the probability of being financially weak. Compute the threshold that should be used if we want to make a classification based on the odds of being financially weak, and the threshold for the corresponding logit

Using the coefficient of total loans&leases to total assets ratio, the odds of being financially weak increases as TotLns&Lses/Assets decreases.

A cut off greater than 0.5 will end up classifying fewer records as poor banks; whereas, a cut off less than 0.5 will end up classifying more records as poor banks.To minimize the expected cost of misclassification, the cutoff value for classification should be decreased.